In [1]:
import numpy as np
import pandas as pd
import mordred
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem.Draw import IPythonConsole

# pd.set_option('display.max_rows', 6000)
# pd.set_option('display.max_columns', 2000)

In [2]:
SOURCE_TABLE_PATH = '../amino.csv'
SOURCE_SDF_PATH = '../amino.sdf'

In [3]:
df_origin = pd.read_csv(SOURCE_TABLE_PATH)

In [4]:
mols = PandasTools.LoadSDF(SOURCE_SDF_PATH).ROMol
print(len(mols))

20


In [5]:
calc = Calculator(descriptors, ignore_3D=False)
descs = calc.pandas(mols)

In [6]:
def convert_error_to_nan(descs):
    for i, col_val in enumerate(descs.values.tolist()):
        for j, val in enumerate(col_val):
            if isinstance(val, mordred.error.Missing):
                descs.iloc[i, j] = np.nan
            elif isinstance(val, bool):
                descs.iloc[i,j] = [0, 1][val]
    return descs

In [7]:
descs = convert_error_to_nan(descs)

In [8]:
descs.isnull().sum().sum()

7324

In [9]:
df = pd.concat([df_origin, descs], axis=1)
df.head()

,CID,IsomericSMILES,y,name,split,ABC,ABCGG,nAcid,nBase,SpAbs_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,5950,C[C@@H](C(=O)O)N,0,alanine,test,3.932653,4.244375,1,1,6.000000,...,7.626083,30.698690,89.047678,6.849821,29,4,22.0,21.0,4.222222,1.444444
1,6322,C(C[C@@H](C(=O)O)N)CN=C(N)N,0,arginine,test,8.284683,8.139582,1,4,13.438278,...,8.423542,40.531488,174.111676,6.696603,247,11,48.0,48.0,6.333333,2.861111
2,6267,C([C@@H](C(=O)O)N)C(=O)N,0,asparagine,test,6.163363,6.515899,1,1,9.335326,...,8.267962,36.359125,132.053492,7.767852,96,8,36.0,36.0,5.583333,2.111111
3,5960,C([C@@H](C(=O)O)N)C(=O)O,1,aspartic acid,test,6.163363,6.515899,2,1,9.335326,...,8.267962,36.359125,133.037508,8.314844,96,8,36.0,36.0,5.583333,2.111111
4,5862,C([C@@H](C(=O)O)N)S,1,cysteine,test,4.530370,5.004088,1,1,7.662988,...,7.890957,32.688753,121.019749,8.644268,46,6,26.0,26.0,4.472222,1.777778


In [10]:
df_train = df[df.split=='train']
df_test = df[df.split=='test']

In [11]:
df_train.to_csv('../train.csv', encoding='utf-8', index=False)
df_test.to_csv('../test.csv', encoding='utf-8', index=False)